In [1]:
import os
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
file_path = "/data/zju-48b/xhl/eval/my_eval/data/math500/test.jsonl"
df = pd.read_json(file_path, lines=True)
problem_list = df["problem"][:20]

slow_thinking = "Walk me through your complete reasoning process step by step. Output as many procedures as you can."
fast_thinking = "Please directly answer the question."

texts = []

for prob in problem_list:
    texts.append(f"""<|im_start|>user
{prob}
{slow_thinking}<|im_end|>
<|im_start|>assistant
That""")

for prob in problem_list:
    texts.append(f"""<|im_start|>user
{prob}
{fast_thinking}<|im_end|>
<|im_start|>assistant
That""")

In [2]:
from vllm import LLM
import easysteer.hidden_states as hs

llm = LLM(
    model=model_path,
    task="embed", 
    tensor_parallel_size=1,
    enforce_eager=True,
    enable_prefix_caching=False,
    enable_chunked_prefill=False
)
all_hidden_states, outputs = hs.get_all_hidden_states(llm, texts)

INFO 11-04 00:29:39 [utils.py:253] non-default args: {'task': 'embed', 'enable_prefix_caching': False, 'disable_log_stats': True, 'enforce_eager': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/'}
INFO 11-04 00:29:39 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-04 00:29:39 [model.py:1746] Using max model len 32768
INFO 11-04 00:29:43 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:44 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:51 [default_loader.py:314] Loading weights took 1.12 seconds
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:51 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:52 [gpu_model_runner.py:2971] Model loading took 2.8876 GiB and 1.278235 seconds
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:54 [gpu_worker.py:343] Available KV cache memory: 37.75 GiB
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:55 [kv_cache_utils.py:1247] GPU KV cache size: 1,413,584 tokens
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:55 [kv_cache_utils.py:1252] Maximum concurrency for 32,768 tokens per request: 43.14x
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:55 [core.py:238] init engine (profile, create kv cache, warmup model) took 2.62 seconds
(EngineCore_DP0 pid=2687648) INFO 11-04 00:29:55 [vllm.py:414] Cudagraph is disabled under eager mode
INFO 11-04 00:29:56 [llm.py:346] Support

Adding requests:   0%|          | 0/40 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/40 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [3]:
from easysteer.steer import extract_pca_control_vector
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(20)), 
    negative_indices=list(range(20,40)),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=True,
    method="center"
)
control_vector.export_gguf("MATH500.gguf")

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

[2025-11-04 00:30:30] INFO pca.py:114: Layer 0: PCA on centered data explains 91.40128% of the variance
[2025-11-04 00:30:30] INFO pca.py:169: Layer 0: Direction corrected (flipped)
[2025-11-04 00:30:30] INFO pca.py:114: Layer 1: PCA on centered data explains 89.63355% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 2: PCA on centered data explains 90.07514% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 3: PCA on centered data explains 89.11943% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 4: PCA on centered data explains 89.48713% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 5: PCA on centered data explains 86.34245% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 6: PCA on centered data explains 83.49532% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 7: PCA on centered data explains 83.14339% of the variance
[2025-11-04 00:30:30] INFO pca.py:114: Layer 8: PCA on centered data explains 81.67219% of